# Visualization of MS spectrum

In [1]:
%matplotlib inline

from pathlib import Path

from matplotlib import pyplot as plt,__version__ as plt_version
from pkg_resources import get_distribution  # Comes with setuptools.
from pyteomics import mzml, auxiliary

import ipywidgets as widgets
from IPython.display import clear_output

print(f"Matplotlib version : {plt_version}")
try:
    print(f"pyteomics version: {get_distribution('pyteomics').version}")
except:
    print("pyteomics version not found")

Matplotlib version : 3.1.1
pyteomics version: 4.1.2


Choosing the file is done programmatically as the file upload may not work due to the size
of the mzML files.

In [2]:
# Get the mzml file.
DATA_DIR = Path("..").resolve().joinpath("data/")

mzml_file = DATA_DIR.joinpath("1937003-Q1.mzML")

In [3]:
scans = []
mz_arrays = []
intensity_arrays = []
retention_times = []

In [ ]:
# Get the MS1 spectra.
with mzml.read(str(mzml_file)) as reader:
    for spectrum in reader:
        if spectrum["ms level"] == 1:
            scans.append(spectrum["id"])
            mz_arrays.append(spectrum["m/z array"])
            intensity_arrays.append(spectrum["intensity array"])
            retention_times.append(spectrum["scanList"]["scan"][0]["scan start time"])

In [ ]:
# Widget management.
file_box = widgets.Select(
    options=retention_times,
    # Writing Retention time is too long for the default label.
    description='RT (minutes):',
    disabled=False,
)

button = widgets.Button(description='Display',
                        button_style="info")


# Configuring the widgets interactions.

out = widgets.Output()

# Display the image when the button is clicked.  Is rather slow.
def display_image(_):
    with out:
        clear_output()
        # Supposes the retention time have same index as arrays.
        spectrum_index = retention_times.index(file_box.value)
        # The width needs to be adjusted for some arrays with extreme peaks.
        plt.bar(mz_arrays[spectrum_index], intensity_arrays[spectrum_index])
        plt.xlabel("m/z")
        plt.ylabel("intensity")
        plt.show()

button.on_click(display_image)

In [ ]:
# Main widget.
box = widgets.VBox([file_box, button, out])
box